<a href="https://colab.research.google.com/github/kimiafa/Network/blob/main/CRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div style="direction: rtl; text-align: right; font-family: Tahoma; font-size: 16px; line-height: 28px;">

# شبیه‌سازی CRC (Cyclic Redundancy Check)

در این نوت‌بوک، الگوریتم CRC برای مدل‌های مختلف پیاده‌سازی و بررسی می‌شود.

## مدل‌های مورد بررسی:
- CRC-8
- CRC-16-IBM
- CRC-32

## اهداف:
1. پیاده‌سازی CRC با پایتون
2. مشاهده خروجی مدل‌های مختلف
3. آزمایش تشخیص خطا با تغییر بیت
4. مقایسه نتایج

</div>

def reflect(value, width):
    result = 0
    for i in range(width):
        if (value >> i) & 1:
            result |= 1 << (width - 1 - i)
    return result

def compute_crc(data, poly, init, xor_out, width, refin=False, refout=False):
    # اگر ورودی string باشد تبدیلش می‌کنیم به bytes
    if isinstance(data, str):
        data = data.encode()

    mask = (1 << width) - 1
    topbit = 1 << (width - 1)

    crc = init & mask

    # حالت LSB-first
    if refin:
        poly_ref = reflect(poly, width)
        for b in data:
            crc ^= b
            for _ in range(8):
                if crc & 1:
                    crc = (crc >> 1) ^ poly_ref
                else:
                    crc >>= 1
                crc &= mask

    else:
        # حالت MSB-first
        for b in data:
            crc ^= (b << (width - 8)) & mask
            for _ in range(8):
                if crc & topbit:
                    crc = ((crc << 1) ^ poly) & mask
                else:
                    crc = (crc << 1) & mask

    # بازتاب خروجی اگر لازم باشد
    if refout:
        crc = reflect(crc, width)

    crc ^= (xor_out & mask)
    return crc & mask

print("CRC-8:", hex(compute_crc("Hello CRC", 0x07, 0x00, 0x00, 8)))
print("CRC-16-IBM:", hex(compute_crc("Hello CRC", 0x8005, 0x0000, 0x0000, 16, refin=True, refout=True)))
print("CRC-32:", hex(compute_crc("Hello CRC", 0x04C11DB7, 0xFFFFFFFF, 0xFFFFFFFF, 32, refin=True, refout=True)))

def compute_crc(data, poly, init, xor_out, width):
    """
    محاسبه CRC با پارامترهای عمومی
    """
    crc = init
    for byte in data:
        crc ^= (byte << (width - 8))
        for _ in range(8):
            if crc & (1 << (width - 1)):
                crc = ((crc << 1) ^ poly) & ((1 << width) - 1)
            else:
                crc = (crc << 1) & ((1 << width) - 1)
    return crc ^ xor_out

<div style="direction: rtl; text-align: right; font-family: Tahoma; font-size: 16px;">

## مدل‌های مورد استفاده در این شبیه‌سازی

سه مدل پرکاربرد CRC در این تمرین پیاده‌سازی شده است.

</div>

crc_models = {
    "CRC-8": {
        "poly": 0x07,
        "init": 0x00,
        "xor": 0x00,
        "width": 8
    },
    "CRC-16-IBM": {
        "poly": 0x8005,
        "init": 0xFFFF,
        "xor": 0x0000,
        "width": 16
    },
    "CRC-32": {
        "poly": 0x04C11DB7,
        "init": 0xFFFFFFFF,
        "xor": 0xFFFFFFFF,
        "width": 32
    }
}

<div style="direction: rtl; text-align: right; font-family: Tahoma; font-size: 16px;">

## محاسبه CRC برای داده ورودی

در این بخش برای رشته `"Hello CRC"` مقدار CRC محاسبه می‌شود.

</div>

data = b"Hello CRC"
results = {}

for name, m in crc_models.items():
    results[name] = hex(
        compute_crc(
            data,
            m["poly"],
            m["init"],
            m["xor"],
            m["width"]
        )
    )

results

<div style="direction: rtl; text-align: right; font-family: Tahoma; font-size: 16px;">

## آزمایش تشخیص خطا

در این تست، مقدار CRC برای داده اصلی و داده‌ای که تنها یک بیت آن تغییر کرده مقایسه می‌شود.

</div>

original = b"OpenAI"
tampered = b"OpenBi"

for name, m in crc_models.items():
    crc1 = compute_crc(original, m["poly"], m["init"], m["xor"], m["width"])
    crc2 = compute_crc(tampered, m["poly"], m["init"], m["xor"], m["width"])
    
    print(f"{name} → original: {hex(crc1)}   tampered: {hex(crc2)}")

<div style="direction: rtl; text-align: right; font-family: Tahoma; font-size: 16px;">

## تست مدل‌ها روی داده‌های مختلف

</div>

samples = [b"test", b"hello", b"123456", b"CRC Check"]

for text in samples:
    print(f"\nInput: {text}")
    for name, m in crc_models.items():
        val = compute_crc(text, m["poly"], m["init"], m["xor"], m["width"])
        print(f"  {name}: {hex(val)}")